In [4]:
import re
import keras
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.layers import *
from keras.models import Sequential
import keras.utils

# Буквы

In [3]:
with open('05 train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '')  # убираем первый невидимый символ
    text = text.replace('\n', ' ')  # заменяем перевод строки на пробел
    text = re.sub(r'[^А-я ]', '', text)  # убираем все недопустимые символы

In [4]:
text

'Вы  лучший ответ на проблемы которые возникли в понедельник Думайте позитивно и верьте в свою способность достигать отличных результатов Если вы смогли в понедельник подняться с постели значит вы супер герой'

In [5]:
num_characters = 34  # 33 буквы + пробел
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [6]:
tokenizer.fit_on_texts(text)
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'т': 3, 'е': 4, 'и': 5, 'в': 6, 'н': 7, 'с': 8, 'л': 9, 'п': 10, 'ь': 11, 'ы': 12, 'р': 13, 'а': 14, 'д': 15, 'у': 16, 'к': 17, 'з': 18, 'ч': 19, 'й': 20, 'м': 21, 'г': 22, 'б': 23, 'я': 24, 'ш': 25, 'ю': 26, 'х': 27}


In [15]:
inp_chars = 7
data = tokenizer.texts_to_matrix(text)

In [16]:
n = data.shape[0]-inp_chars

In [17]:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:]  # предсказание следующего символа

In [18]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 500)               267500    
                                                                 
 dense_1 (Dense)             (None, 34)                17034     
                                                                 
Total params: 284534 (1.09 MB)
Trainable params: 284534 (1.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
7/7 [==============================] - 1s 15ms/step - loss: 3.3952 - accuracy: 0.0700
Epoch 2/100
7/7 [==============================] - 0s 19ms/step - loss: 2.3937 - accuracy: 0.3250
Epoch 3/100
7/7 [==============================] - 0s 20ms/step - loss: 1.8742 - accuracy: 0.5400
Epoch 4/100
7/7 [==============================] - 0s 14ms/step - loss: 1.5052 - accuracy: 0.5650
Epoch 5/100
7/7 [==============================] - 0s 14ms/step - loss: 1.1793 - accuracy: 0.6750
Epoch 6/100
7/7 [==============================] - 0s 16ms/step - loss: 0.9856 - accuracy: 0.7150
Epoch 7/100
7/7 [==============================] - 0s 15ms/step - loss: 0.8157 - accuracy: 0.8150
Epoch 8/100
7/7 [==============================] - 0s 15ms/step - loss: 0.7076 - accuracy: 0.7950
Epoch 9/100
7/7 [==============================] - 0s 16ms/step - loss: 0.6185 - accuracy: 0.8550
Epoch 10/100
7/7 [==============================] - 0s 17ms/step - loss: 0.5430 - accuracy: 0.8550
Epoch 11/100
7/7 [=

In [20]:
def buildPharse(inp_str, str_len=50):
    for i in range(str_len):
        x=[]
        for j in range(i, i+inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в OneHot encoding
        x=np.array(x)
        inp=x.reshape(1, inp_chars,num_characters)
        pred=model.predict(inp) # предсказываем ОНЕ четвёртого символа
        d=tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении
        inp_str+=d # дописываем строку
    return inp_str

In [21]:
res=buildPharse('проблем')
print(res)

1/1 [==============================] - 0s 15ms/step
проблемы которые возникли в понедельник подняться с посте


увеличив количество букв, на основании которых сеть даёт предсказание - получили почти осмысленный текст

In [24]:
res=buildPharse('лучший ответ')
print(res)

1/1 [==============================] - 0s 15ms/step
лучший ответответ на писа проблеистлемы кои глоторыпотопо возн


Уже менее осмысленно

In [26]:
res=buildPharse('Думайте позитивно')
print(res)

1/1 [==============================] - 0s 18ms/step
Думайте позитивно позитивно и вернвно и верьте в   ерьте в своюдеен


In [27]:
res=buildPharse('он герой')
print(res)

1/1 [==============================] - 0s 15ms/step
он геройното сы свооелиплвначиос еы втонери линою е меробт


Уже совсем не то.<br>
Вероятно надо больший объём тренирочного текста.<br>
Ну и более сложную сеть.

# Слова

In [28]:
with open('05 train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ
    texts = texts.replace('\n', ' ')  # заменяем перевод строки на пробел
    texts = texts.lower()
    texts = re.sub(r'[^А-я ]', '', texts)  # убираем все недопустимые символы
texts

'Вы  лучший ответ на проблемы которые возникли в понедельник Думайте позитивно и верьте в свою способность достигать отличных результатов Если вы смогли в понедельник подняться с постели значит вы супер герой'

In [29]:
maxWordsCount=1000

In [30]:
tokenizerWord = Tokenizer(num_words=maxWordsCount, char_level=False)
tokenizerWord.fit_on_texts([texts])

In [31]:
dist=list(tokenizerWord.word_counts.items())
print(dist)

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1), ('позитивно', 1), ('и', 1), ('верьте', 1), ('свою', 1), ('способность', 1), ('достигать', 1), ('отличных', 1), ('результатов', 1), ('если', 1), ('смогли', 1), ('подняться', 1), ('с', 1), ('постели', 1), ('значит', 1), ('супер', 1), ('герой', 1)]


In [33]:
data=tokenizerWord.texts_to_sequences([texts])
res=keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print(res.shape)

(31, 1000)


In [34]:
res

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [36]:
inp_words = 3
n = res.shape[0]-inp_words
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]  # предсказание следующего слова

In [37]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_2 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
1/1 [==============================] - 1s 737ms/step - loss: 2.5896 - accuracy: 0.3571
Epoch 2/50
1/1 [==============================] - 0s 13ms/step - loss: 2.5272 - accuracy: 0.3571
Epoch 3/50
1/1 [==============================] - 0s 12ms/step - loss: 2.4667 - accuracy: 0.3929
Epoch 4/50
1/1 [==============================] - 0s 14ms/step - loss: 2.4057 - accuracy: 0.4643
Epoch 5/50
1/1 [==============================] - 0s 12ms/step - loss: 2.3435 - accuracy: 0.5357
Epoch 6/50
1/1 [==============================] - 0s 15ms/step - loss: 2.2806 - accuracy: 0.6071
Epoch 7/50
1/1 [==============================] - 0s 14ms/step - loss: 2.2175 - accuracy: 0.6429
Epoch 8/50
1/1 [==============================] - 0s 12ms/step - loss: 2.1553 - accuracy: 0.6786
Epoch 9/50
1/1 [==============================] - 0s 12ms/step - loss: 2.0947 - accuracy: 0.7500
Epoch 10/50
1/1 [==============================] - 0s 13ms/step - loss: 2.0362 - accuracy: 0.7857
Epoch 11/50
1/1 [===========

In [42]:
def buildPharseWords(text, str_len=4):
    res=text
    data=tokenizerWord.texts_to_sequences(text)
    # так как получаем data вида [[],[],[3],[]], то есть часть элементов имеют нулевую длину, а значит неопределены
    # потому впишем туда значение 0
    for j in range(len(data)):
      if len(data[j])==0:
        data[j].append(0)
    for i in range(str_len):
      x=keras.utils.to_categorical(data[i:i+inp_words], num_classes=maxWordsCount) # преобразуем слова в OneHot encoding
      inp=x.reshape(1, inp_words,maxWordsCount)
      pred=model.predict(inp) # предсказываем ОНЕ четвёртого слова
      indx=pred.argmax(axis=1)[0]
      data.append([indx])
      res+=' '+dist[indx][0] # дописываем строку - достаём из списка кортежей по индексу значение слова
    return res

In [43]:
res=buildPharseWords('голубые слоны мавритании')
print(res)


1/1 [==============================] - 0s 15ms/step
голубые слоны мавритании достигать понедельник верьте результатов


In [46]:
res=buildPharseWords('лучший ответ на проблемы')
print(res)

1/1 [==============================] - 0s 14ms/step
лучший ответ на проблемы достигать отличных герой позитивно


косноязычный текст - так как малый набор слов в словарном запасе сети.

In [48]:
data=tokenizerWord.texts_to_sequences([texts])
res=keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
inp_words = 6
n = res.shape[0]-inp_words
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]  # предсказание следующего слова
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(512, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, batch_size=32, epochs=30)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 512)               774656    
                                                                 
 dense_3 (Dense)             (None, 1000)              513000    
                                                                 
Total params: 1287656 (4.91 MB)
Trainable params: 1287656 (4.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1/1 [==============================] - 1s 827ms/step - loss: 6.9262 - accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 28ms/step - loss: 6.6382 - accuracy: 0.7200
Epoch 3/30
1/1 [==============================] - 0s 26ms/step - loss: 6.3367 - accuracy: 1.0000
Epoch 4/30
1/1 [==============================] - 0s 28ms/step - loss: 5.9928 - accuracy: 1.0000
Epoch 5/30
1/1 [===

In [53]:
def buildPharseWords(text, str_len=4):
    res=text
    data=tokenizerWord.texts_to_sequences(text)
    # так как получаем data вида [[],[],[3],[]], то есть часть элементов имеют нулевую длину, а значит неопределены
    # потому впишем туда значение 0
    for j in range(len(data)):
      if len(data[j])==0:
        data[j].append(0)
    for i in range(str_len):
      x=keras.utils.to_categorical(data[i:i+inp_words], num_classes=maxWordsCount) # преобразуем слова в OneHot encoding
      inp=x.reshape(1, inp_words,maxWordsCount)
      pred=model.predict(inp) # предсказываем ОНЕ четвёртого слова
      indx=pred.argmax(axis=1)[0]
      data.append([indx])
      res+=' '+dist[indx][0] # дописываем строку - достаём из списка кортежей по индексу значение слова
    return res

In [54]:
res=buildPharseWords('голубые слоны мавритании')
print(res)

1/1 [==============================] - 0s 16ms/step
голубые слоны мавритании супер лучший свою ответ


In [63]:
def buildPharseWords(text, str_len=25):
    res=text
    data=tokenizerWord.texts_to_sequences(text)
    # так как получаем data вида [[],[],[3],[]], то есть часть элементов имеют нулевую длину, а значит неопределены
    # потому впишем туда значение 0
    for j in range(len(data)):
      if len(data[j])==0:
        data[j].append(0)
    for i in range(str_len):
      x=keras.utils.to_categorical(data[i:i+inp_words], num_classes=maxWordsCount) # преобразуем слова в OneHot encoding
      inp=x.reshape(1, inp_words,maxWordsCount)
      pred=model.predict(inp) # предсказываем ОНЕ четвёртого слова
      indx=pred.argmax(axis=1)[0]
      data.append([indx])
      print(i,len(data), indx)
      res+=' '+dist[indx][0] # дописываем строку - достаём из списка кортежей по индексу значение слова
    return res

In [68]:
res=buildPharseWords('Вы  лучший ответ на проблемы которые возникли')
print(res)

1/1 [==============================] - 0s 23ms/step
0 46 10
1/1 [==============================] - 0s 27ms/step
1 47 17
1/1 [==============================] - 0s 23ms/step
2 48 9
1/1 [==============================] - 0s 24ms/step
3 49 23
1/1 [==============================] - 0s 22ms/step
4 50 13
1/1 [==============================] - 0s 24ms/step
5 51 2
1/1 [==============================] - 0s 22ms/step
6 52 22
1/1 [==============================] - 0s 21ms/step
7 53 2
1/1 [==============================] - 0s 20ms/step
8 54 3
1/1 [==============================] - 0s 27ms/step
9 55 24
1/1 [==============================] - 0s 27ms/step
10 56 9
1/1 [==============================] - 0s 23ms/step
11 57 2
1/1 [==============================] - 0s 22ms/step
12 58 14
1/1 [==============================] - 0s 26ms/step
13 59 10
1/1 [==============================] - 0s 25ms/step
14 60 2
1/1 [==============================] - 0s 22ms/step
15 61 25
1/1 [==============================] - 0s

Из-за малого словарного запаса, ни увеличение нейронов сети, ни увеличение количества слов-оснований предсказания заметно не улучшают качество выдаваемого текста

In [7]:
with open('текст.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    # texts = text.replace('\ufeff', '')  # убираем первый невидимый символ
    texts = texts.replace('\n', ' ')  # заменяем перевод строки на пробел
    texts = texts.lower()
    texts = re.sub(r'[^А-я ]', '', texts)  # убираем все недопустимые символы

In [9]:
maxWordsCount=1000
tokenizerWord = Tokenizer(num_words=maxWordsCount, char_level=False)
tokenizerWord.fit_on_texts([texts])
data=tokenizerWord.texts_to_sequences([texts])
res=keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
inp_words = 6
n = res.shape[0]-inp_words
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]  # предсказание следующего слова
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(512, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, batch_size=32, epochs=30)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 512)               774656    
                                                                 
 dense (Dense)               (None, 1000)              513000    
                                                                 
Total params: 1287656 (4.91 MB)
Trainable params: 1287656 (4.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1529/1529 [==============================] - 37s 24ms/step - loss: 5.7329 - accuracy: 0.0573
Epoch 2/30
1529/1529 [==============================] - 36s 24ms/step - loss: 5.4227 - accuracy: 0.0809
Epoch 3/30
1529/1529 [==============================] - 37s 24ms/step - loss: 5.1132 - accuracy: 0.1037
Epoch 4/30
1529/1529 [==============================] - 39s 25ms/step - loss: 4.7692 - accuracy: 0.118

In [10]:
dist=list(tokenizerWord.word_counts.items())
print(len(dist))

18075


In [13]:
def buildPharseWords(text, str_len=10):
    res=text
    data=tokenizerWord.texts_to_sequences(text)
    # так как получаем data вида [[],[],[3],[]], то есть часть элементов имеют нулевую длину, а значит неопределены
    # потому впишем туда значение 0
    for j in range(len(data)):
      if len(data[j])==0:
        data[j].append(0)
    for i in range(str_len):
      x=keras.utils.to_categorical(data[i:i+inp_words], num_classes=maxWordsCount) # преобразуем слова в OneHot encoding
      inp=x.reshape(1, inp_words,maxWordsCount)
      pred=model.predict(inp) # предсказываем ОНЕ четвёртого слова
      indx=pred.argmax(axis=1)[0]
      data.append([indx])
      res+=' '+dist[indx][0] # дописываем строку - достаём из списка кортежей по индексу значение слова
    return res

In [16]:
res=buildPharseWords('голубые слоны мавритании')
print(res)

1/1 [==============================] - 0s 15ms/step
голубые слоны мавритании магистр шхиян магистр звезд магистр магистр магистр может анонс магистр


In [15]:
res=buildPharseWords('лучший ответ на проблемы которые возникли')
print(res)

1/1 [==============================] - 0s 15ms/step
лучший ответ на проблемы которые возникли шхиян звезд магистр магистр середину из магистр шхиян магистр врата


In [14]:
res=buildPharseWords('его жену по приказу царя отправили в дальний монастырь')
print(res)

1/1 [==============================] - 0s 15ms/step
его жену по приказу царя отправили в дальний монастырь сергей магистр году магистр магистр хотел магистр всего сергей сергей


Большой словарный запас и глубина отслеживания слов =6 как-то не добавили особо смысла получаемого текста...
Хотя - модель не дообучилась - 30 эпох оказалось мало - процесс увеличения точности, как и уменьшения потерь шёл уверенно и равномерно.

In [18]:
inp_words = 3
data=tokenizerWord.texts_to_sequences([texts])
res=keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
n = res.shape[0]-inp_words
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]  # предсказание следующего слова
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, batch_size=32, epochs=30)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_1 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1529/1529 [==============================] - 9s 5ms/step - loss: 5.7037 - accuracy: 0.0630
Epoch 2/30
1529/1529 [==============================] - 7s 5ms/step - loss: 5.3647 - accuracy: 0.0972
Epoch 3/30
1529/1529 [==============================] - 8s 5ms/step - loss: 5.0767 - accuracy: 0.1191
Epoch 4/30
1529/1529 [==============================] - 8s 5ms/step - loss: 4.8104 - accuracy: 0.1367
Epoch 

In [19]:
res=buildPharseWords('голубые слоны мавритании')
print(res)

1/1 [==============================] - 0s 21ms/step
голубые слоны мавритании сергей пикник шхиян шхиян сергей сергей магистр сергей новых сергей


In [20]:
res=buildPharseWords('лучший ответ на проблемы которые возникли')
print(res)

1/1 [==============================] - 0s 16ms/step
лучший ответ на проблемы которые возникли шхиян шхиян сергей шхиян анонс сергей сергей невозможно магистр сергей


In [22]:
dist[:20]

[('черный', 5),
 ('магистр', 57),
 ('сергей', 1),
 ('шхиян', 1),
 ('анонс', 1),
 ('он', 610),
 ('хотел', 31),
 ('всего', 50),
 ('лишь', 6),
 ('съездить', 2),
 ('на', 1140),
 ('пикник', 1),
 ('но', 374),
 ('врата', 1),
 ('времени', 69),
 ('отворились', 1),
 ('и', 2594),
 ('забросили', 1),
 ('его', 303),
 ('в', 1839)]

Чего она так к сергею шхияну привязалась?!

In [23]:
res=buildPharseWords('его жену по приказу царя отправили в дальний монастырь')
print(res)

1/1 [==============================] - 0s 15ms/step
его жену по приказу царя отправили в дальний монастырь сергей сергей сергей сергей сергей пикник шхиян шхиян сергей сергей


В целом - полный бред.

In [24]:
history = model.fit(X, Y, batch_size=256, epochs=30)

Epoch 1/30
192/192 [==============================] - 3s 16ms/step - loss: 2.0460 - accuracy: 0.5376
Epoch 2/30
192/192 [==============================] - 3s 15ms/step - loss: 2.0020 - accuracy: 0.5511
Epoch 3/30
192/192 [==============================] - 3s 15ms/step - loss: 1.9896 - accuracy: 0.5555
Epoch 4/30
192/192 [==============================] - 3s 17ms/step - loss: 1.9821 - accuracy: 0.5567
Epoch 5/30
192/192 [==============================] - 3s 15ms/step - loss: 1.9770 - accuracy: 0.5574
Epoch 6/30
192/192 [==============================] - 3s 15ms/step - loss: 1.9719 - accuracy: 0.5583
Epoch 7/30
192/192 [==============================] - 3s 16ms/step - loss: 1.9658 - accuracy: 0.5590
Epoch 8/30
192/192 [==============================] - 3s 17ms/step - loss: 1.9623 - accuracy: 0.5589
Epoch 9/30
192/192 [==============================] - 3s 15ms/step - loss: 1.9570 - accuracy: 0.5612
Epoch 10/30
192/192 [==============================] - 3s 15ms/step - loss: 1.9513 - accura

In [25]:
res=buildPharseWords('его жену по приказу царя отправили в дальний монастырь')
print(res)

1/1 [==============================] - 0s 16ms/step
его жену по приказу царя отправили в дальний монастырь магистр сергей сергей магистр магистр пикник анонс шхиян магистр сергей


Что-то там не то с архитектурой сети.

In [27]:
inp_words = 3
data=tokenizerWord.texts_to_sequences([texts])
res=keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
n = res.shape[0]-inp_words
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]  # предсказание следующего слова
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, batch_size=64, epochs=10)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_2 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
765/765 [==============================] - 6s 7ms/step - loss: 5.7495 - accuracy: 0.0591
Epoch 2/10
765/765 [==============================] - 5s 6ms/step - loss: 5.4840 - accuracy: 0.0872
Epoch 3/10
765/765 [==============================] - 5s 7ms/step - loss: 5.2451 - accuracy: 0.1083
Epoch 4/10
765/765 [==============================] - 5s 6ms/step - loss: 5.0257 - accuracy: 0.1245
Epoch 5/10
765

In [28]:
res=buildPharseWords('его жену по приказу царя отправили в дальний монастырь')
print(res)

1/1 [==============================] - 0s 14ms/step
его жену по приказу царя отправили в дальний монастырь может его сергей сергей сергей пикник шхиян сергей может его


Уменьшение количества эпох обучения - тоже не слишком добавило разнообразия предлагаемым словам